# Joom

In [6]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')


os.chdir(r'D:\Joom\Final Raw data')

date = pd.to_datetime('today').strftime('_%d%m%Y_%H_%M')

data = pd.read_excel('MMM_dataset_2021-09-15.xlsx',sheet_name='data_2021-09-16 04_42_41 PM')
data['Year'] = data['partition_date'].dt.year

df = data.melt(id_vars=data.columns[data.columns.isin(['partition_date','os_type','Year'])])

df['os_var'] = df['variable']+'_'+df['os_type']
df['os_var_yr'] = df['os_var']+'_'+df['Year'].astype('str')
df.variable.unique()
#split 2020 into Pre march & post march (29th March 2020)
#split weekday & weekend

# points used, coupons given & coupons value
# agg_list = {i:(np.where(i in ['dau','loyal_users','new_users'],np.mean,np.sum).tolist()) for i in df.variable}


array(['new_users', 'clicks_google', 'spend_google', 'clicks_partners',
       'spend_partners', 'clicks_organic', 'users_with_coupons',
       'orders_with_coupons', 'discount', 'revenue', 'orders',
       'users_with_points', 'orders_with_points', 'points_used',
       'reviews_1', 'reviews_2', 'reviews_3', 'reviews_4', 'reviews_5',
       'loyal_users', 'dau'], dtype=object)

In [2]:
res1 = df.pivot_table(index='partition_date',columns='os_var',values='value')
res2 = data.groupby(['partition_date','Year']).agg(agg_list)
res3 = pd.merge(res1,res2,right_index=True,left_index=True)

res01 = df.pivot_table(index='partition_date',columns='os_var_yr',values='value')
res4 = pd.merge(res3,res01,right_index=True,left_index=True)

var_yr = res2.reset_index().melt(id_vars=res2.reset_index().columns[:2])
var_yr['var_yr'] = var_yr['variable']+'_'+var_yr['Year'].astype('str')
res03 = var_yr.pivot_table(index='partition_date',columns='var_yr',values='value')
res5 = pd.merge(res4,res03,right_index=True,left_index=True).reset_index()

# Facebook

In [3]:
os.chdir(r'D:\Joom\fb\Latest')

files = os.listdir(os.getcwd())
files = [i for i in files if '.csv' in i]

data = {}
for i in files:
    data[i] = pd.read_csv(i,parse_dates = ['Date'])
data = pd.concat(data,0,ignore_index=True)
data= data[data['Date'].dt.year.between(2019,2021)]

data['Account ID name'] = np.where(data['Account ID']=="'466301817475915", 'Retargeting', 'Prospecting')
data['Year'] = data['Date'].dt.year

result = data.pivot_table(index=['Account ID name','Platform','Campaign Objective', 'Ad Set Name', 'Placement' ,'Video','Date','Year'],
                        values=['Impressions','Spend'],aggfunc='sum').reset_index()

result['Ad Set Name'] = result['Ad Set Name'].str.lower()


result['Campaign'] = np.where(result['Video']==True , 'Video Views',
                              np.where(result['Ad Set Name'].str.contains('android'),'Android_Static'+"_"+result['Account ID name'],
                                       np.where(result['Ad Set Name'].str.contains('ios'),'iOS_Static'+"_"+result['Account ID name'],
                                                'Unclassified_Static'+"_"+result['Account ID name'])))

result['Campaign_yr'] = result['Campaign'] +'_'+ result['Year'].astype('str')


r1 = result.pivot_table(columns='Campaign',index='Date',values='Impressions',aggfunc='sum')
r2 = result.pivot_table(columns='Campaign_yr',index='Date',values='Impressions',aggfunc='sum')
r3 = pd.merge(r1,r2,left_index=True,right_index=True).reset_index()
r3.rename(columns={'Date':'partition_date'},inplace=True)

In [4]:
final = pd.merge(res5,r3,on='partition_date',how='left').fillna(0)
final.to_excel(f'Joom_datacube{date}.xlsx')